# 2: Data Cleaning

In [42]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np
from src.data_processing import (
    load_grant_data, 
    clean_currency, 
    categorize_program,
    validate_data_quality
)

import warnings
warnings.filterwarnings('ignore')

## Load Raw Data

In [43]:
# Using our helper function
df = load_grant_data('../data/raw/MCHB_Data_GranteeDetails.csv')
print(f"Loaded {len(df)} records")
df.head()

Loaded 4946 records


,Program Area,Program Name,Fiscal Year,Activity Code,Grant Number,Grantee Name,State,County,Congressional District,Grantee Class,Awardee Amount,Grantee Contact Details
0,Maternal and Child Health,Healthy Start Initiative-Eliminating Racial/Et...,2021,H49,H49MC27827,Birmingham Healthy Start Plus Inc,AL,Jefferson,AL-06,"Corporate Entity, Federal Tax Exempt",1269121.0,"1105 52nd St S Birmingham, AL 35222-3924"
1,Maternal and Child Health,Maternal and Child Health Nutrition Training P...,2021,T79,T79MC00011,University Of Alabama At Birmingham,AL,Jefferson,AL-07,Other,225000.0,"701 20th St S Birmingham, AL 35233-2031"
2,Maternal and Child Health,Leadership Education in Adolescent Health (LEAH),2021,T71,T71MC24209,University Of Alabama At Birmingham,AL,Jefferson,AL-07,Other,437942.0,"701 20th St S Birmingham, AL 35233-2031"
3,Maternal and Child Health,Leadership Education in Neurodevelopmental and...,2021,T73,T73MC00003,University Of Alabama At Birmingham,AL,Jefferson,AL-07,Other,713466.0,"701 20th St S Birmingham, AL 35233-2031"
4,Maternal and Child Health,Pediatric Pulmonary Centers,2021,T72,T72MC00001,University Of Alabama At Birmingham,AL,Jefferson,AL-07,Other,340000.0,"701 20th St S Birmingham, AL 35233-2031"


## Fix Currency Formatting

In [44]:
# The load function already cleaned this, but let's verify
print("Sample of cleaned amounts:")
print(df[['Awardee Amount']].head(10))

# Check for any remaining non-numeric values
non_numeric = df[df['Awardee Amount'].isna()]
print(f"\nRows with invalid amounts: {len(non_numeric)}")

Sample of cleaned amounts:
   Awardee Amount
0       1269121.0
1        225000.0
2        437942.0
3        713466.0
4        340000.0
5       1150000.0
6         96750.0
7        985898.0
8        130000.0
9      11523951.0

Rows with invalid amounts: 0


## Handle Missing Values

In [45]:
# Check missing values again
missing_summary = pd.DataFrame({
    'Missing': df.isnull().sum(),
    'Percent': (df.isnull().sum() / len(df) * 100).round(2)
})
print(missing_summary[missing_summary['Missing'] > 0])

Empty DataFrame
Columns: [Missing, Percent]
Index: []


In [46]:
# before and after missing values

print(f"Records before dropping: {len(df)}")
df_clean = df.dropna(subset=['Awardee Amount'])
print(f"Records after dropping: {len(df_clean)}")
print(f"Dropped {len(df) - len(df_clean)} rows with missing amounts")

Records before dropping: 4946
Records after dropping: 4946
Dropped 0 rows with missing amounts


## Standardize State Names

In [47]:
# Check current state values
print("Unique states/territories:")
print(sorted(df_clean['State'].unique()))
print(f"\nTotal: {df_clean['State'].nunique()}")

Unique states/territories:
['AK', 'AL', 'AR', 'AS', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'FM', 'GA', 'GU', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MH', 'MI', 'MN', 'MO', 'MP', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'PR', 'PW', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VI', 'VT', 'WA', 'WI', 'WV', 'WY']

Total: 59


In [48]:
df_clean['State'] = df_clean['State'].str.upper().str.strip()

## Create Program Categories

In [49]:
# Using categorization function
df_clean['program_category'] = df_clean['Program Name'].apply(categorize_program)

print("Program categories created:")
print(df_clean['program_category'].value_counts())

Program categories created:
program_category
Maternal Health             1676
Other                       1250
Training & Education         595
Emergency Services           445
Screening Programs           422
Special Healthcare Needs     377
Mental Health                114
Home Visiting                 67
Name: count, dtype: int64


## Handle Outliers

In [50]:
# Identify outliers using IQR method
Q1 = df_clean['Awardee Amount'].quantile(0.25)
Q3 = df_clean['Awardee Amount'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_clean['is_outlier'] = (
    (df_clean['Awardee Amount'] < lower_bound) | 
    (df_clean['Awardee Amount'] > upper_bound)
)

print(f"Outliers identified: {df_clean['is_outlier'].sum()}")
print(f"\nOutlier bounds: ${lower_bound:,.0f} to ${upper_bound:,.0f}")

Outliers identified: 562

Outlier bounds: $-1,144,309 to $2,242,426


In [51]:
# Look at sample of outliers
print("Sample of outlier grants:")
outliers = df_clean[df_clean['is_outlier']]
print(outliers[['State', 'Program Name', 'Awardee Amount']].sort_values('Awardee Amount', ascending=False).head(10))

Sample of outlier grants:
     State                        Program Name  Awardee Amount
2643    CA  Maternal and Child Health Services      39725961.0
1812    CA  Maternal and Child Health Services      39660787.0
1006    CA  Maternal and Child Health Services      39383025.0
86      CA  Maternal and Child Health Services      39255586.0
3451    CA  Maternal and Child Health Services      39113905.0
4229    CA  Maternal and Child Health Services      39040391.0
1443    NY  Maternal and Child Health Services      38406749.0
582     NY  Maternal and Child Health Services      38366219.0
3089    NY  Maternal and Child Health Services      38121968.0
2266    NY  Maternal and Child Health Services      38085393.0


## Create Additional Useful Columns

In [52]:
# Fiscal year as int
df_clean['fiscal_year'] = df_clean['Fiscal Year'].astype(int)

# Binary flag for mental health programs
df_clean['is_mental_health'] = df_clean['program_category'] == 'Mental Health'

# Binary flag for maternal health
df_clean['is_maternal_health'] = df_clean['program_category'] == 'Maternal Health'

# Log transform of amount for modeling
df_clean['log_amount'] = np.log10(df_clean['Awardee Amount'])

print("New columns created:")
print(df_clean[['program_category', 'is_mental_health', 'is_maternal_health', 'log_amount']].head())

New columns created:
       program_category  is_mental_health  is_maternal_health  log_amount
0       Maternal Health             False                True    6.103503
1       Maternal Health             False                True    5.352183
2  Training & Education             False               False    5.641417
3  Training & Education             False               False    5.853373
4                 Other             False               False    5.531479


## Final Data Quality Check

In [53]:
validate_data_quality(df_clean)

DATA QUALITY REPORT

Total records: 4,946
Total columns: 18

--- Missing Values ---
Empty DataFrame
Columns: [Missing, Percent]
Index: []

--- Duplicates ---
Duplicate rows: 0

--- Grant Amounts ---
Min: $1.00
Max: $39,725,961.00
Mean: $1,473,590.64
Median: $372,000.00



In [54]:
# Check for duplicates
dups = df_clean.duplicated(subset=['Grant Number']).sum()
print(f"Duplicate grant numbers: {dups}")

if dups > 0:
    # Remove duplicates if any
    df_clean = df_clean.drop_duplicates(subset=['Grant Number'], keep='first')
    print(f"Removed {dups} duplicates")

Duplicate grant numbers: 3107
Removed 3107 duplicates


## Save Cleaned Data

In [55]:
# Save to processed folder
output_path = '../data/processed/mchb_grants_cleaned.csv'
df_clean.to_csv(output_path, index=False)

print(f"Saved {len(df_clean)} cleaned records to {output_path}")
print(f"\nColumns in cleaned dataset: {len(df_clean.columns)}")
print(f"File size: {df_clean.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

Saved 1839 cleaned records to ../data/processed/mchb_grants_cleaned.csv

Columns in cleaned dataset: 18
File size: 1.57 MB
